In [2]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
import tensorflow as tf
from PIL import Image
import os
import cv2
import torch
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from keras.models import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# model trainen zonder attributes

In [3]:
labels = np.load("feather-in-focus/class_names.npy", allow_pickle=True).item()
df = pd.read_csv("feather-in-focus/train_images.csv")
df

,image_path,label
0,/train_images/1.jpg,1
1,/train_images/2.jpg,1
2,/train_images/3.jpg,1
3,/train_images/4.jpg,1
4,/train_images/5.jpg,1
...,...,...
3921,/train_images/3922.jpg,200
3922,/train_images/3923.jpg,200
3923,/train_images/3924.jpg,200
3924,/train_images/3925.jpg,200


In [4]:
attributes = np.load("feather-in-focus/attributes.npy")
len(attributes)


200

In [5]:
att_list = []
for i in range(len(df)):
    att_list.append(attributes[df['label'][i]-1])  
    
df['attributes'] = att_list
# df = df.drop('image_path', axis=1)
df

,image_path,label,attributes
0,/train_images/1.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
1,/train_images/2.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
2,/train_images/3.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
3,/train_images/4.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
4,/train_images/5.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
...,...,...,...
3921,/train_images/3922.jpg,200,"[0.04378018711713792, 0.02814440600394273, 0.0..."
3922,/train_images/3923.jpg,200,"[0.04378018711713792, 0.02814440600394273, 0.0..."
3923,/train_images/3924.jpg,200,"[0.04378018711713792, 0.02814440600394273, 0.0..."
3924,/train_images/3925.jpg,200,"[0.04378018711713792, 0.02814440600394273, 0.0..."


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Load CSV files
train_df = pd.read_csv("feather-in-focus/train_images.csv")
train_df['label'] = train_df['label'] - 1
df['label'] = df['label'] - 1


def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = cv2.imread(f'feather-in-focus/train_images{img_path}' )
        img = cv2.resize(img, (224, 224))  # Resize the image
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
#     return np.array(images)
    return images

# Load and preprocess images
train_images = load_and_preprocess_images(train_df['image_path'])

In [ ]:
df['pre_image'] = train_images
df

,image_path,label,attributes,pre_image
0,/train_images/1.jpg,0,"[0.010638400403539122, 0.010638400403539122, 0...","[[[0.6078431372549019, 0.3843137254901961, 0.1..."
1,/train_images/2.jpg,0,"[0.010638400403539122, 0.010638400403539122, 0...","[[[0.6588235294117647, 0.6235294117647059, 0.5..."
2,/train_images/3.jpg,0,"[0.010638400403539122, 0.010638400403539122, 0...","[[[0.8274509803921568, 0.796078431372549, 0.79..."
3,/train_images/4.jpg,0,"[0.010638400403539122, 0.010638400403539122, 0...","[[[0.6352941176470588, 0.6196078431372549, 0.6..."
4,/train_images/5.jpg,0,"[0.010638400403539122, 0.010638400403539122, 0...","[[[0.7098039215686275, 0.7137254901960784, 0.6..."
...,...,...,...,...
3921,/train_images/3922.jpg,199,"[0.04378018711713792, 0.02814440600394273, 0.0...","[[[0.3568627450980392, 0.3568627450980392, 0.3..."
3922,/train_images/3923.jpg,199,"[0.04378018711713792, 0.02814440600394273, 0.0...","[[[0.3411764705882353, 0.36470588235294116, 0...."
3923,/train_images/3924.jpg,199,"[0.04378018711713792, 0.02814440600394273, 0.0...","[[[0.043137254901960784, 0.13725490196078433, ..."
3924,/train_images/3925.jpg,199,"[0.04378018711713792, 0.02814440600394273, 0.0...","[[[0.3137254901960784, 0.39215686274509803, 0...."


In [8]:
# Load the InceptionV3 model with pre-trained weights
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

#For now, we will freeze the model layers
base_model.trainable = False

# Create a Sequential model
model = tf.keras.Sequential()

# Add the base model (InceptionV3) to the Sequential model
model.add(base_model)

# Global Average Pooling layer
model.add(tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer"))

#More hidden layers, now with BatchNormalization
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))


model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))


# Output Dense layer
model.add(tf.keras.layers.Dense(200, activation="softmax", name="output-layer"))

# As we allow the model to be trainable in some layers, it will be better to decrease the learning rate in this region
# to avoid an exagerated change in the model weights 

base_learning_rate = 0.01 

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)

# Compile the model 
model.compile(
    loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"]
)

87910968/87910968 [==============================] - 3s 0us/step


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling_laye  (None, 2048)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_94 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               1

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame with columns 'image' and 'label'.
# 'image' column contains the image arrays and 'label' column contains class labels.

# Split your data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert 'image' column to numpy array
train_images = np.stack(train_df['pre_image'].to_numpy())
val_images = np.stack(val_df['pre_image'].to_numpy())

# Convert labels to categorical
train_labels = tf.keras.utils.to_categorical(train_df['label'])
val_labels = tf.keras.utils.to_categorical(val_df['label'])

# Create datasets
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(batch_size)

# Example: Check the shape of the datasets
for images, labels in train_dataset.take(1):
    print("Training batch shape:", images.shape, labels.shape)

for images, labels in val_dataset.take(1):
    print("Validation batch shape:", images.shape, labels.shape)


Training batch shape: (64, 224, 224, 3) (64, 200)
Validation batch shape: (64, 224, 224, 3) (64, 200)


In [ ]:
#Setting the early_stop to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=3,
    min_delta=0.001,
    restore_best_weights=True,)


history = model.fit(
    train_dataset,
    epochs=18,
    steps_per_epoch=len(train_dataset),
    validation_data=val_dataset,
    validation_steps=int(0.25 * len(val_dataset)),
#     callbacks=[early_stop],
)

Epoch 1/18
43/43 [==============================] - 50s 1s/step - loss: 4.5483 - accuracy: 0.0921 - val_loss: 22.6118 - val_accuracy: 0.0391
Epoch 2/18
43/43 [==============================] - 44s 1s/step - loss: 3.2038 - accuracy: 0.2405 - val_loss: 6.1927 - val_accuracy: 0.1797
Epoch 3/18
43/43 [==============================] - 42s 989ms/step - loss: 2.6069 - accuracy: 0.3359 - val_loss: 4.3931 - val_accuracy: 0.2344
Epoch 4/18
43/43 [==============================] - 42s 986ms/step - loss: 2.0939 - accuracy: 0.4316 - val_loss: 3.8064 - val_accuracy: 0.2695
Epoch 5/18
43/43 [==============================] - 42s 968ms/step - loss: 1.8126 - accuracy: 0.4836 - val_loss: 3.8836 - val_accuracy: 0.2656
Epoch 6/18
43/43 [==============================] - 42s 969ms/step - loss: 1.5990 - accuracy: 0.5360 - val_loss: 3.7777 - val_accuracy: 0.3203
Epoch 7/18
43/43 [==============================] - 43s 998ms/step - loss: 1.3821 - accuracy: 0.6052 - val_loss: 3.8014 - val_accuracy: 0.2852
Epoc

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  311


In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 260

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

    # Now that we will allow some layers to be unfreezed, it's better to decrease the learning rate to avoid dramatic changes in those 
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100)

# Compile the model again
model.compile(
    loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"]
)

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=4,
    min_delta=0.001,
    restore_best_weights=True,)


fine_tune_epochs = 12
total_epochs =  18 + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         steps_per_epoch=len(train_dataset),
                         validation_data=val_dataset,
                         validation_steps=int(0.25 * len(val_dataset)),
                         callbacks=[early_stop])

Epoch 9/30


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000291FC31F400>
Traceback (most recent call last):
  File "C:\Users\maxvt\AppData\Local\Programs\Python\Python310\lib\weakref.py", line 371, in remove
    self = selfref()
KeyboardInterrupt: 


50/50 [==============================] - 150s 1s/step - loss: 1.3820 - accuracy: 0.6076 - val_loss: 2.5304 - val_accuracy: 0.3750
Epoch 10/30
50/50 [==============================] - 49s 983ms/step - loss: 1.3068 - accuracy: 0.6127 - val_loss: 2.4098 - val_accuracy: 0.4010
Epoch 11/30
50/50 [==============================] - 47s 942ms/step - loss: 1.2290 - accuracy: 0.6338 - val_loss: 2.3571 - val_accuracy: 0.4010
Epoch 12/30
50/50 [==============================] - 49s 976ms/step - loss: 1.1798 - accuracy: 0.6583 - val_loss: 2.3330 - val_accuracy: 0.4062
Epoch 13/30
50/50 [==============================] - 48s 962ms/step - loss: 1.1659 - accuracy: 0.6697 - val_loss: 2.3108 - val_accuracy: 0.4010
Epoch 14/30
50/50 [==============================] - 48s 951ms/step - loss: 1.1533 - accuracy: 0.6640 - val_loss: 2.2982 - val_accuracy: 0.4167
Epoch 15/30
50/50 [==============================] - 47s 946ms/step - loss: 1.1234 - accuracy: 0.6758 - val_loss: 2.2888 - val_accuracy: 0.4219
Epoch 

In [ ]:
# Save the model
model.save('model.h5')
model.save('model.keras')

In [ ]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model('path_to_save_model.keras')